# 连接到 Galaxy

In [1]:
from bioblend.galaxy import GalaxyInstance

import yaml
import json


# 从配置文件读取设置
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

gi = GalaxyInstance(
    url=config['galaxy_url'],
    key=config['api_key']
)

print(gi)

GalaxyInstance object for Galaxy at http://aimedorig.natapp1.cc/transmolecule


# datasets, histories, libraries

## 查看所有历史面板记录

In [4]:
# 获取历史记录
histories = gi.histories.get_histories()

print(len(histories))
if len(histories) > 0:
#     print(histories[0])
    print(json.dumps(histories[0], indent=4, ensure_ascii=False))

    # 打印所有历史记录信息
    print("\nid\tname\tcount(items)\tupdate_time")
    for h in histories:
        print(f"{h['id']}\t{h['name']}\t{h['count']}\t{h['update_time']}")

40
{
    "model_class": "History",
    "id": "319886f37b7797fe",
    "name": "12-4-tteatkkk",
    "deleted": false,
    "purged": false,
    "archived": false,
    "url": "/api/histories/319886f37b7797fe",
    "published": false,
    "count": 14,
    "annotation": null,
    "tags": [],
    "update_time": "2025-12-04T09:14:40.322656",
    "preferred_object_store_id": null
}

id	name	count(items)	update_time
319886f37b7797fe	12-4-tteatkkk	14	2025-12-04T09:14:40.322656
f3a63b25ef201245	12-4-tteat	15	2025-12-04T09:12:38.481177
6f91353f3eb0fa4a	12-4-ttea	4	2025-12-04T08:58:00.926086
84ffe6fca6c4fbda	test_history1	0	2025-12-04T03:26:24.152205
f1b9846ab84237e7	test_history	21	2025-12-04T08:38:06.873712
6d4c8d216c52b289	kk	1	2025-12-04T02:59:42.965217
12e4b4feedfe9f3f	test_history	0	2025-12-04T01:40:49.659771
c851ab275e52f8af	Unnamed history	0	2025-12-04T01:40:49.484089
16e792953f5b45db	tool_run_test	17	2025-12-03T08:56:14.197923
be0a27b9edd0db03	tool_run_test	17	2025-12-03T08:53:22.751243
819

In [ ]:
history_recent = gi.histories.get_most_recently_used_history()
print(history_recent)
print(history_recent['id'])

## 查看历史记录及数据

In [ ]:
# 查看历史记录信息
history_id = '5114a2a207b7caff'

# contents=False （默认值），则只会得到历史记录中包含的数据集的 ID 列表
# contents=True ，则会得到每个数据集的元数据
history_info = gi.histories.show_history(history_id, contents=False)
print(json.dumps(history_info, indent=4, ensure_ascii=False))

# 查看历史记录面板所有数据
history_info = gi.histories.show_history(history_id, contents=True)
print(len(history_info))
print(type(history_info))
if len(history_info) > 0:
    print("id\thid\tdeleted\tname\tcreate_time")
    for h in history_info:
        print(f"{h['id']}\t{h['hid']}\t{h['deleted']}\t{h['name']}\t{h['create_time']}")

In [ ]:
# 查看数据
content_id = '2a314cb8e047cf03'

content_info = gi.datasets.show_dataset(content_id)
print(json.dumps(content_info, indent=4, ensure_ascii=False))

## history 新建、上传文件、删除

In [ ]:
new_history = gi.histories.create_history("kk")
print(new_history)

new_history_id = new_history['id']

# 设置 web 默认打开的 history
# gi.histories.open_history(new_history_id)

# 上传文件
res = gi.tools.upload_file('data/test.json', new_history_id, file_type='json')
print(res)

In [ ]:
res = gi.tools.upload_file('data/test.json', new_history_id, file_type='json')
print(res)

In [ ]:
# 永久删除历史记录
res = gi.histories.delete_history(new_history_id, purge=True)
print(res)

## Tools

In [ ]:
# 获取工具信息
tools = gi.tools.get_tool_panel()
# print(tools)

peptide_design = None
print('id\t name\t description')
for tool_section in tools:
    for tool in tool_section['elems']:
        print(f'{tool["id"]}\t {tool["name"]}\t {tool["description"]}')
    print()
    
    if tool_section['id'] == 'peptide_design':
        peptide_design = tool_section
        
print(json.dumps(peptide_design, indent=4, ensure_ascii=False))

In [ ]:
tool_id = gi.tools.get_tools(name='PharmaPepGen')[0]['id']
tool_info = gi.tools.show_tool(tool_id)
print(tool_info)

# history_id 任意 id 都可以，打印工具需要的输入数据格式(查看工具输入格式)
tool_info = gi.tools.build(tool_id=tool_id, history_id=history_id, tool_version='1.0')
print(json.dumps(tool_info['inputs'], indent=4, ensure_ascii=False))

In [ ]:
tool_info = gi.tools.build(tool_id='pharma', history_id=history_id, tool_version='1.0')
print(json.dumps(tool_info['inputs'], indent=4, ensure_ascii=False))

### 运行工具

In [ ]:
# 创建一个新的记录
new_history = gi.histories.create_history("tool_run_test")
new_history_id = new_history['id']

# 上传文件
json_file = gi.tools.upload_file('data/test.json', new_history_id, file_type='json')
json_file_id = json_file['outputs'][0]['id']

# tool
tool_id = gi.tools.get_tools(name='PharmaPepGen')[0]['id']

# "src": "hda"
# hda = History Dataset Association 普通历史里的数据集
# ldda ＝ Library Dataset Dataset Association（数据文库里的文件）
# hdca ＝ History Dataset Collection Association（历史里的数据集集合）
# ld ＝ Library Dataset（旧式文库条目）
input_info = {
    "input_files": [{"id": json_file_id, "src": "hda"}],
    "num_samples": 15
}

job = gi.tools.run_tool(history_id=new_history_id, tool_id=tool_id, tool_inputs=input_info)
print(json.dumps(job, indent=4, ensure_ascii=False))

# 打开浏览器
# gi.histories.open_history(new_history_id)

In [ ]:
# 下载数据
output_collections_id = job['output_collections'][0]['collection_id']
output_file = "kk2.zip"

_res = gi.dataset_collections.download_dataset_collection(output_collections_id, output_file)
print(_res)

# Workflows

In [2]:
workflows = gi.workflows.get_workflows()

print(len(workflows))
if len(workflows) > 0:
    print(json.dumps(workflows[0], indent=4, ensure_ascii=False))

    print("\nid\tname\tupdate_time")
    for w in workflows:
        print(f"{w['id']}\t{w['name']}\t{w['update_time']}")

4
{
    "model_class": "StoredWorkflow",
    "id": "33b43b4e7093c91f",
    "name": "peptide_design",
    "create_time": "2025-12-05T05:19:30.790807",
    "update_time": "2025-12-05T05:23:21.111361",
    "published": false,
    "importable": false,
    "deleted": false,
    "hidden": false,
    "tags": [],
    "latest_workflow_uuid": "7b3f06db-5946-4be3-877d-9dc2fb06e8a7",
    "annotations": [
        ""
    ],
    "url": "*deprecated attribute, URL not filled in by server*",
    "owner": "yudd",
    "source_metadata": null,
    "number_of_steps": 4,
    "show_in_tool_panel": false
}

id	name	update_time
33b43b4e7093c91f	peptide_design	2025-12-05T05:23:21.111361
1cd8e2f6b131e891	PDE1	2025-11-27T06:11:02.966382
f2db41e1fa331b3e	Workflow constructed from history 'PDE'	2025-11-27T06:02:02.630584
f597429621d6eb2b	Workflow constructed from history 'PDE'	2025-07-15T16:06:59.097908


In [3]:
workflow_id = '33b43b4e7093c91f'
workflow_info = gi.workflows.show_workflow(workflow_id)
print(json.dumps(workflow_info, indent=4, ensure_ascii=False))

{
    "model_class": "StoredWorkflow",
    "id": "33b43b4e7093c91f",
    "name": "peptide_design",
    "create_time": "2025-12-05T05:19:30.790807",
    "update_time": "2025-12-05T05:23:21.111361",
    "published": false,
    "importable": false,
    "deleted": false,
    "hidden": false,
    "tags": [],
    "latest_workflow_uuid": "7b3f06db-5946-4be3-877d-9dc2fb06e8a7",
    "url": "/transmolecule/api/workflows/33b43b4e7093c91f",
    "owner": "yudd",
    "email_hash": "c348eeefe7be7eff5e8da7639a3bd99c",
    "slug": "peptidedesign",
    "inputs": {
        "0": {
            "label": "Receptor",
            "value": "",
            "uuid": "08e741b8-df06-40da-b7f9-989f49548b0f"
        },
        "1": {
            "label": "Ligand",
            "value": "",
            "uuid": "7cae0ccd-9d45-44ac-84c1-c75f65b8d671"
        }
    },
    "annotation": "",
    "license": null,
    "creator": null,
    "source_metadata": null,
    "steps": {
        "0": {
            "id": 0,
            "

# call

In [ ]:
from transmolecule import TransMolecule
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# 登录
trans_molecule = TransMolecule(config['galaxy_url'], config['api_key'])

# 创建一个新的历史纪录
trans_molecule.history.create(name='test_history')

# 上传数据到当前历史记录
data = trans_molecule.tool.upload(file_dir='./data/test_1_pharma_pharmapepgen')

print(data)

In [ ]:
from transmolecule import TransMolecule
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# 登录
trans_molecule = TransMolecule(config['galaxy_url'], config['api_key'])

# 创建一个新的历史纪录
trans_molecule.history.select('f1b9846ab84237e7')
trans_molecule.history.info()
data = trans_molecule.history.content()
print(data)



# Job

In [6]:
# 打印当前所有job
history_id = gi.histories.get_most_recently_used_history()['id']
print(history_id)

jobs = gi.jobs.get_jobs(history_id=history_id)
print(jobs)


319886f37b7797fe
[{'model_class': 'Job', 'id': '26c6e36433226a2c', 'state': 'ok', 'exit_code': 0, 'update_time': '2025-12-04T09:14:40.325146', 'create_time': '2025-12-04T09:14:35.192325', 'galaxy_version': '23.1', 'tool_id': 'pharma_pep_gen', 'history_id': '319886f37b7797fe', 'external_id': '11230', 'command_line': 'mkdir input_dir && mkdir output_dir && cp -r "/opt/app/transmolecule/galaxy/tools/docker/4_1_peptide_design/pharma_pep_gen"/* . && file_item="Pharma Res(json)" && file_name=$(echo ${file_item} | sed \'s/\\..*//\') && file_name=$(echo ${file_item} | sed \'s/ //\') && cp /opt/app/transmolecule/galaxy/database/objects/8/0/b/dataset_80b333d1-4889-42fc-a545-25a6be140e55.dat ./input_dir/${file_name}.json &&  /opt/miniconda3/envs/diffu_peptide/bin/python predict_on_targets.py --json_dir ./input_dir --model_path ./fine_tuned_best.pt --output_dir ./output_dir --num_samples 10', 'traceback': None}, {'model_class': 'Job', 'id': 'b0645819402ce636', 'state': 'ok', 'exit_code': 0, 'updat

In [9]:
job_id = 'b0645819402ce636'
job_id = '26c6e36433226a2c'
inputs = gi.jobs.get_inputs(job_id=job_id)
print(inputs)

[{'name': 'input_files', 'dataset': {'src': 'hda', 'id': '15081ab54a789ee2'}}, {'name': 'input_files1', 'dataset': {'src': 'hda', 'id': '15081ab54a789ee2'}}]


In [10]:
res = gi.jobs.get_outputs(job_id=job_id)
print(res)

[{'name': '__new_primary_file_output|PharmaRes(json)_sample_1__', 'dataset': {'src': 'hda', 'id': '6e0f198cc0c22322'}}, {'name': '__new_primary_file_output|PharmaRes(json)_sample_10__', 'dataset': {'src': 'hda', 'id': 'a048e6f4c3b90e64'}}, {'name': '__new_primary_file_output|PharmaRes(json)_sample_2__', 'dataset': {'src': 'hda', 'id': '38494c41afc4e63a'}}, {'name': '__new_primary_file_output|PharmaRes(json)_sample_3__', 'dataset': {'src': 'hda', 'id': '680b17e947950440'}}, {'name': '__new_primary_file_output|PharmaRes(json)_sample_4__', 'dataset': {'src': 'hda', 'id': '3238011cf6e619ce'}}, {'name': '__new_primary_file_output|PharmaRes(json)_sample_5__', 'dataset': {'src': 'hda', 'id': 'd3018d65969b4a89'}}, {'name': '__new_primary_file_output|PharmaRes(json)_sample_6__', 'dataset': {'src': 'hda', 'id': 'ddb80a62b83a4438'}}, {'name': '__new_primary_file_output|PharmaRes(json)_sample_7__', 'dataset': {'src': 'hda', 'id': '183b606e8dac647b'}}, {'name': '__new_primary_file_output|PharmaRes(

In [11]:
res = gi.jobs.show_job(job_id=job_id)
print(res)

{'model_class': 'Job', 'id': '26c6e36433226a2c', 'state': 'ok', 'exit_code': 0, 'update_time': '2025-12-04T09:14:40.325146', 'create_time': '2025-12-04T09:14:35.192325', 'galaxy_version': '23.1', 'command_version': None, 'copied_from_job_id': None, 'tool_id': 'pharma_pep_gen', 'history_id': '319886f37b7797fe', 'external_id': '11230', 'command_line': 'mkdir input_dir && mkdir output_dir && cp -r "/opt/app/transmolecule/galaxy/tools/docker/4_1_peptide_design/pharma_pep_gen"/* . && file_item="Pharma Res(json)" && file_name=$(echo ${file_item} | sed \'s/\\..*//\') && file_name=$(echo ${file_item} | sed \'s/ //\') && cp /opt/app/transmolecule/galaxy/database/objects/8/0/b/dataset_80b333d1-4889-42fc-a545-25a6be140e55.dat ./input_dir/${file_name}.json &&  /opt/miniconda3/envs/diffu_peptide/bin/python predict_on_targets.py --json_dir ./input_dir --model_path ./fine_tuned_best.pt --output_dir ./output_dir --num_samples 10', 'traceback': None, 'params': {'num_samples': '"10"', 'chromInfo': '"/op

In [12]:
res = gi.jobs.get_state(job_id=job_id)
print(res)

ok
